### --- Assume all company submitted; assume all blank columns and length of 1; assume "0.2x size" format wont change

In [136]:
import pandas as pd
import numpy as np
import xlrd

In [137]:
sheets_dict = pd.read_excel("Input_File.xlsx", sheet_name=None)
workbook = xlrd.open_workbook("Input_File.xlsx")

In [138]:
sheet_names = workbook.sheet_names()
worksheets = {}
for i in range(len(sheet_names)-1):
    worksheets[sheet_names[i+1]] = workbook.sheet_by_name(sheet_names[i+1])

In [139]:
def check_not_empty(worksheet,row,col):
        return worksheet.cell(row,col).value != ''

def check_whole_col_empty(worksheet,tot_rows,col):
    result = True
    for i in range(tot_rows):
        if check_not_empty(worksheet,i,col):
            result = False
            break;
    return result;

def combine_consecutive_number(alist):
    seat_need_delete = []
    for i in range(len(alist)-1):
        if alist[i+1] == alist[i]+1:
            seat_need_delete.append(alist[i])
    result = list(set(alist)-set(seat_need_delete))
    result.sort()
    return result

def str_contains(str1,str2,str3='yuezhou',str4='yuezhou'):
    result = False
    if str2 in str1:
        result = True
    if str3 in str1:
        result = True
    if str4 in str1:
        result = True
    return result;

def get_size_from_string(astring):
    astring = astring.replace('x','')
    return float(astring.split()[0])

### Get all blank rows and cols

In [140]:
worksheet_now = worksheets[sheet_names[1]]
n_cols = worksheet_now.ncols
n_rows = worksheet_now.nrows

effective_cols =[]
effective_rows = []

for i in range(n_cols):
    if not check_whole_col_empty(worksheet_now,n_rows,i):
        effective_cols.append(i)
        
empty_cols = list(set(range(0,n_cols))-set(effective_cols))
empty_cols.sort()        

# attention: first 2 in the effective cols are "median" and "bis/ask spread for a trade"
for i in range(n_rows):
    if check_not_empty(worksheet_now,i,empty_cols[0]+1):
        effective_rows.append(i)
        
above_table_empty_rows = list(set(range(effective_rows[0],effective_rows[-1]))-set(effective_rows))
above_table_empty_rows.sort()
above_table_empty_rows = combine_consecutive_number(above_table_empty_rows)

### get CCY --list

In [141]:
currency = worksheet_now.cell(4,0).value
currency

'USD'

### Get Trade Type --list

In [142]:
tradetype = []
for i in above_table_empty_rows:
    # no need to plus 1 here
    tradetype.append(worksheet_now.cell(i+1,0).value)

### Get Firm name --list

In [143]:
firms = []
for i in range(n_cols):
    if check_not_empty(worksheet_now,above_table_empty_rows[0]-1,i):
        firms.append(worksheet_now.cell(above_table_empty_rows[0]-1,i).value)

###  tenors (and sizes) -- a dict  indexing on Trade Type --- and is same for each company

In [145]:
# different trade type may have different ternor collection
tenors = {}
size_bases = {}
frame_rows = above_table_empty_rows.copy()
frame_cols = empty_cols.copy()
frame_rows.append(effective_rows[-1]+1)
frame_cols.append(effective_cols[-1]+1)
j=0
# first loop: each descriptive table:higher to lower

for i in range(len(frame_rows)-1): 
    
    symbol = '+'
    row_inuse = frame_rows[i]
    next_row = frame_rows[i+1]
    tenor_ele = {}
    tenor_choice = []
    size_choice = []
    
    name = tradetype[j]
    
    for rows_in_table in range(row_inuse+2,next_row):
        a_tenor = ''
        a_size = 0
        if rows_in_table == row_inuse+2:
            for k in range(1,empty_cols[0]):
                if check_not_empty(worksheet_now,rows_in_table,k):
                    tenor_ele[k] = worksheet_now.cell(rows_in_table,k).value
                    
        if rows_in_table != row_inuse+2:    
            for k in range(1,empty_cols[0]): # empty is in fact the furtherest 
                if check_not_empty(worksheet_now,rows_in_table,k):
                    if  worksheet_now.cell(rows_in_table,k).value >0:
                        symbol = '+'
                    else:
                        symbol = '-'
                    a_tenor= a_tenor + symbol + tenor_ele[k]+ ' '
                    a_size = worksheet_now.cell(rows_in_table,k).value
            tenor_choice.append(a_tenor)
            size_choice.append(abs(a_size)*1000000)
    tenors[name] = tenor_choice  
    size_bases[name] = size_choice
    j=j+1

### size multiplier; which indexing on tradetype

In [193]:
size_mul = {}
for i in above_table_empty_rows:
    multiplier = []
    name = worksheet_now.cell(i+1,0).value
    for j in range(empty_cols[0]+1,empty_cols[1]):
        if check_not_empty(worksheet_now,i+2,j):
            multiplier.append(get_size_from_string(worksheet_now.cell(i+2,j).value))
            #multiplier.append(worksheet_now.cell(i+2,j).value)
        else:pass;
    size_mul[name] = multiplier

In [194]:
size_mul

{'Basis Swap': [0.2, 0.5, 1.0, 2.0, 5.0, 10.0],
 'Fed Fund - LIBOR Basis Swap': [0.2, 0.5, 1.0, 2.0, 5.0, 10.0],
 'Invoice Swap': [0.2, 0.5, 1.0, 2.0, 5.0, 10.0],
 'OIS Outright': [0.2, 0.5, 1.0, 2.0, 5.0, 10.0],
 'Vanilla Butterfly': [0.2, 0.5, 1.0, 2.0, 5.0, 10.0],
 'Vanilla Curve Spread': [0.2, 0.5, 1.0, 2.0, 5.0, 10.0],
 'Vanilla Outright': [0.2, 0.5, 1.0, 2.0, 5.0, 10.0],
 'Zero Coupon Outright': [0.2, 0.5, 1.0, 2.0, 5.0, 10.0]}

### loop each company to find out the spread

In [146]:
output = pd.DataFrame(columns= ['CCY','Trade_Type','Firm','Tenor','DV01_Size','Bid_Ask_Spread'])

In [147]:
frame_rows

[6, 23, 36, 49, 60, 73, 80, 93, 106]

In [148]:
frame_cols

[8,
 15,
 22,
 29,
 36,
 43,
 50,
 57,
 64,
 71,
 78,
 85,
 92,
 99,
 106,
 113,
 120,
 127,
 134,
 141,
 148,
 155,
 162,
 169,
 176]

In [166]:
spreads = []
for j in range(len(frame_rows)-1):    
    row_inuse = frame_rows[j]
    #print(row_inuse)
    row_next = frame_rows[j+1]

    for i in range(len(frame_cols)-1):
        col_inuse = frame_cols[i]+1
        col_next = frame_cols[i+1]
        for arow in range(row_inuse+3,row_next):
            for acol in range(col_inuse,col_next):
                spreads.append(worksheet_now.cell(arow,acol).value)
                #print(arow,acol)
                

### prepare sizes

In [200]:
sizes = {}
for name in tradetype:
    l1 = size_bases[name]
    l2 = size_mul[name]
    sizes[name] = list(map(lambda x ,y: x * y, l1, l2))

In [204]:
#for ccy in sheet_names:
test=0
for tt in tradetype:
    for firm in firms:
        for ten in tenors[tt]:
            for siz in sizes[tt]:
                test = test+1

In [202]:
len(spreads)

10944

In [205]:
test

10752

In [207]:
len(firms)

24

In [208]:
len(tt)

20

In [216]:
tenors

{'Basis Swap': ['+1Y ',
  '+2Y ',
  '+5Y ',
  '+10Y ',
  '+30Y ',
  '+1Y ',
  '+2Y ',
  '+5Y ',
  '+10Y ',
  '+30Y '],
 'Fed Fund - LIBOR Basis Swap': ['+2Y ', '+5Y ', '+10Y ', '+30Y '],
 'Invoice Swap': ['+2Y ',
  '+5Y ',
  '+10Y ',
  '+30Y ',
  '-2Y ',
  '-5Y ',
  '-10Y ',
  '-30Y '],
 'OIS Outright': ['+1Y ',
  '+2Y ',
  '+5Y ',
  '+10Y ',
  '+30Y ',
  '-1Y ',
  '-2Y ',
  '-5Y ',
  '-10Y ',
  '-30Y '],
 'Vanilla Butterfly': ['+1Y -2Y +5Y ',
  '+1Y -2Y +10Y ',
  '+1Y -5Y +10Y ',
  '+1Y -2Y +30Y ',
  '+1Y -5Y +30Y ',
  '+1Y -10Y +30Y ',
  '+2Y -5Y +10Y ',
  '+5Y -10Y +30Y ',
  '+2Y -5Y +30Y ',
  '+2Y -10Y +30Y '],
 'Vanilla Curve Spread': ['+1Y -2Y ',
  '+1Y -5Y ',
  '+1Y -10Y ',
  '+1Y -30Y ',
  '+2Y -5Y ',
  '+2Y -10Y ',
  '+2Y -30Y ',
  '+5Y -10Y ',
  '+5Y -30Y ',
  '+10Y -30Y '],
 'Vanilla Outright': ['+3m** ',
  '+6m** ',
  '+1Y ',
  '+2Y ',
  '+5Y ',
  '+10Y ',
  '+30Y ',
  '-3m** ',
  '-6m** ',
  '-1Y ',
  '-2Y ',
  '-5Y ',
  '-10Y ',
  '-30Y '],
 'Zero Coupon Outright': ['+1Y 

In [213]:
for i in tenors:
    print(len(i))

16
20
17
12
12
27
10
20


In [214]:
for i in sizes:
    print(len(i))

16
20
17
12
12
27
10
20
